Эксперименты с fusion

In [7]:
import re
from unicodedata import category

import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Set
from collections import Counter
import Levenshtein
import matplotlib.pyplot as plt

from src import future_extract as fe


import matplotlib as mpl
import seaborn as sns
mpl.style.use('ggplot')
sns.set_style('white')

In [8]:
data_raw_full = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_train.csv')
data_val_full = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv')

data_raw = data_raw_full[['id', 'description', 'brand_name', 'CommercialTypeName4', 'resolution']]
data_val = data_val_full[['id', 'description', 'brand_name', 'CommercialTypeName4']]

data1 = data_raw.copy(deep = True)

data_cleaner = [data1, data_val]

print (data_raw.info())

data_raw.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197198 entries, 0 to 197197
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id                   197198 non-null  int64 
 1   description          171138 non-null  object
 2   brand_name           116667 non-null  object
 3   CommercialTypeName4  197198 non-null  object
 4   resolution           197198 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 7.5+ MB
None


,id,description,brand_name,CommercialTypeName4,resolution
169012,151961,Дисплей - одна из самых важных частей в смартф...,Wiko,Дисплеи для телефонов,0
86751,1776,Картридж совместимый CLP-K300A Black для Samsu...,Hi-Black,"Картридж, чернила, тонер",0
91101,135073,Аккумулятор для колонки JBL XTREME первой верс...,takoci,Аккумуляторы,0
146803,364527,Huawei matepad pro 11 (GOT) Магнитная втулка M...,MyPads,Чехол для планшетного компьютера,0
118367,203810,Внимание! Перед заказом сравните фото товара с...,NaN,Запчасти для телевизора,0
22525,214412,<p>Мы используем только НОВЫЕ комплектующие пр...,Levsha kaluga,Настольный компьютер,0
29335,302720,"Прочные, изысканные часы с GPS-приемником для ...",Garmin,SmartWatch,0
114125,318991,Смартфон Infinix NOTE 30 выделяется изящным ко...,Infinix,Смартфон,0
117574,294497,Шлейф для Samsung SM-S921B (Galaxy S24) на дис...,Promise Mobile,Шлейфы для телефонов,0
3450,224104,Благодаря своим успешным кампаниям и несметным...,NaN,Видеоигра,0


In [17]:
print(f'Duplicates in train set: {data_raw.duplicated().sum()}, ({np.round(100*data_raw.duplicated().sum()/len(data_raw),1)}%)')
print('')
print(f'Duplicates in test set: {data_val.duplicated().sum()}, ({np.round(100*data_val.duplicated().sum()/len(data_val),1)}%)')

Duplicates in train set: 0, (0.0%)

Duplicates in test set: 0, (0.0%)


In [18]:
print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')

Train columns with null values:
 id                         0
description            26060
brand_name             80531
CommercialTypeName4        0
resolution                 0
dtype: int64
----------
Test/Validation columns with null values:
 id                        0
description            2224
brand_name             5832
CommercialTypeName4       0
dtype: int64
----------


,id,description,brand_name,CommercialTypeName4,resolution
count,197198.000000,171138,116667,197198,197198.000000
unique,NaN,114781,4066,634,NaN
top,NaN,<p>Мы используем только НОВЫЕ комплектующие пр...,iQZiP,Дисплеи для телефонов,NaN
freq,NaN,3009,6084,15002,NaN
mean,225331.179292,NaN,NaN,NaN,0.066187
std,130672.598024,NaN,NaN,NaN,0.248610
min,0.000000,NaN,NaN,NaN,0.000000
25%,112168.250000,NaN,NaN,NaN,0.000000
50%,225326.000000,NaN,NaN,NaN,0.000000
75%,336923.250000,NaN,NaN,NaN,0.000000


In [19]:
data_raw.nunique()

id                     197198
description            114781
brand_name               4066
CommercialTypeName4       634
resolution                  2
dtype: int64

In [20]:
data_raw.dtypes

id                      int64
description            object
brand_name             object
CommercialTypeName4    object
resolution              int64
dtype: object

In [23]:
data_raw.groupby('brand_name')['resolution'].apply(lambda x: sum(x)).sort_values(ascending=False)

brand_name
Sony                600
Mivis               436
Logitech G          299
Pioneer             167
Philips Sonicare    131
                   ... 
Hyundai               0
I LOVE MY             0
I am PC               0
I&Y ELECTRONICS       0
шиппинг & я           0
Name: resolution, Length: 4066, dtype: int64

In [24]:
data_raw.groupby('CommercialTypeName4')['resolution'].apply(lambda x: sum(x)).sort_values(ascending=False)

CommercialTypeName4
Наушники Bluetooth                                884
Внешний аккумулятор                               672
Геймпад для игровой приставки                     560
Типовая батарейка                                 530
Карта памяти                                      479
                                                 ... 
Комплектующие для алкогольного оборудования         0
Комплект чехол + защитное стекло для смартфона      0
Комплект для сборки                                 0
Компактная кухонная плита                           0
Этнический музыкальный инструмент                   0
Name: resolution, Length: 634, dtype: int64

In [54]:
data_raw.groupby(['brand_name', 'CommercialTypeName4'])['resolution'].sum().fillna(0).sort_values(ascending=False)

brand_name     CommercialTypeName4          
Sony           Игровая приставка Playstation    243
Logitech G     Мышь                             153
Sony           Джойстик                         149
               Геймпад для игровой приставки    146
Pioneer        Кондиционер настенный            123
                                               ... 
House&VA       Руль для игровой приставки         0
House Service  Фильтр для пылесосов               0
               Пылесборник                        0
               Насадка для пылесоса               0
шиппинг & я    Коврик для мыши                    0
Name: resolution, Length: 9114, dtype: int64

In [57]:
data_raw.groupby(['CommercialTypeName4', 'brand_name'])['resolution'].sum().fillna(0).sort_values(ascending=False)

CommercialTypeName4                brand_name    
Игровая приставка Playstation      Sony              243
Мышь                               Logitech G        153
Джойстик                           Sony              149
Геймпад для игровой приставки      Sony              146
Кондиционер настенный              Pioneer           123
                                                    ... 
Запчасти для планшета              Promise Mobile      0
                                   ORIGberry           0
                                   OEM                 0
                                   MyPads              0
Этнический музыкальный инструмент  ДУХ ВАРГАНА         0
Name: resolution, Length: 9114, dtype: int64

In [29]:
data_raw['brand_name'].value_counts()

brand_name
iQZiP            6084
ProFDetali       4811
OEM              4275
Sony             3053
Levsha kaluga    3009
                 ... 
Pioneer Woman       1
4CarMedia           1
Trion Worlds        1
AlexAndAlice        1
LANGTU              1
Name: count, Length: 4066, dtype: int64

In [8]:
for df in data_cleaner:
    df['description'] = df['description'].fillna('')
    df['brand_name'] = df['brand_name'].fillna('Unknown')

/var/folders/x1/d4wrwp8x1cdgx44hq2rdb2xr0000gn/T/ipykernel_1296/2986922620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['description'].fillna('')
/var/folders/x1/d4wrwp8x1cdgx44hq2rdb2xr0000gn/T/ipykernel_1296/2986922620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand_name'] = df['brand_name'].fillna('Unknown')


In [9]:
"""
Оптимизированный экстрактор признаков для e-commerce описаний
"""

import re
import pandas as pd
import numpy as np
from typing import Dict, Optional, Set, List
from collections import Counter
from functools import lru_cache
import unicodedata
import string


class OptimizedEcommerceExtractor:
    """
    Оптимизированное извлечение признаков для e-commerce
    """
    
    def __init__(self):
        # Предкомпилированные паттерны (компилируем один раз)
        self._compile_patterns()
        
        # Загружаем внешние данные
        self._load_external_data()
        
        # Кэш для часто используемых операций
        self._cache = {}
        
    def _compile_patterns(self):
        """Предкомпиляция всех regex паттернов для производительности"""
        # HTML паттерны
        self.patterns = {
            # Структурные теги
            'br_tags': re.compile(r'<br\s*/?>', re.IGNORECASE),
            'p_tags': re.compile(r'<p[^>]*>', re.IGNORECASE),
            'div_tags': re.compile(r'<div[^>]*>', re.IGNORECASE),
            'span_tags': re.compile(r'<span[^>]*>', re.IGNORECASE),
            
            # Форматирование
            'strong_tags': re.compile(r'<(?:strong|b)\b[^>]*>', re.IGNORECASE),
            'italic_tags': re.compile(r'<(?:em|i)\b[^>]*>', re.IGNORECASE),
            'underline_tags': re.compile(r'<u\b[^>]*>', re.IGNORECASE),
            
            # Стили
            'font_tags': re.compile(r'<font[^>]*>', re.IGNORECASE),
            'color_attrs': re.compile(r'(?:color|style)\s*=\s*["\'][^"\']*(?:red|green|blue|yellow|#[0-9a-f]{6})', re.IGNORECASE),
            'style_attrs': re.compile(r'style\s*=\s*["\'][^"\']*["\']', re.IGNORECASE),
            
            # Заголовки
            'h_tags': re.compile(r'<h[1-6]\b[^>]*>', re.IGNORECASE),
            'h1_tags': re.compile(r'<h1\b[^>]*>', re.IGNORECASE),
            
            # Entities и прочее
            'html_entities': re.compile(r'&(?:#\d+|#x[0-9a-f]+|\w+);', re.IGNORECASE),
            'script_tags': re.compile(r'<script\b[^>]*>', re.IGNORECASE),
            'style_tags': re.compile(r'<style\b[^>]*>', re.IGNORECASE),
            'broken_tags': re.compile(r'<[^>]*<|>[^<]*>'),
            
            # Подозрительные паттерны
            'multiple_br': re.compile(r'(?:<br\s*/?>[\s]*){3,}', re.IGNORECASE),
            'multiple_bold': re.compile(r'(?:<(?:strong|b)>.*?</(?:strong|b)>[\s]*){3,}', re.IGNORECASE),
            'bright_colors': re.compile(r'style\s*=\s*["\'][^"\']*(?:red|yellow|#ff0000|#ffff00|#00ff00)', re.IGNORECASE),
            'multiple_exclamation': re.compile(r'<[^>]*>.*?!!!+.*?</[^>]*>'),
            'empty_tags': re.compile(r'<(?:p|div|span|strong|b|em|i)(?:\s[^>]*)?>[\s]*</(?:p|div|span|strong|b|em|i)>', re.IGNORECASE),
            
            # Общие паттерны
            'price': re.compile(r'\d+\s*(?:руб|рубл|₽|р\.|dollars?|\$|евро|€)'),
            'clean_html': re.compile(r'<[^>]+>'),
            'sentences': re.compile(r'[.!?]+'),
        }
        
        # Устаревшие теги (множество для быстрой проверки)
        self.deprecated_tags = frozenset(['font', 'center', 'marquee', 'blink'])
        
    def _load_external_data(self):
        """Загрузка внешних данных и словарей"""
        # Ключевые слова копий
        self.copy_keywords = frozenset([
            'копия', 'копию', 'аналог', 'заменитель', 'совместим',
            'подходит', 'альтернатива', 'replica', 'copy', 'compatible',
            'replacement', 'substitute', 'как оригинал', 'точная копия'
        ])
        
        # Призывы к действию
        self.cta_phrases = frozenset([
            'купи', 'закаж', 'спеши', 'скидк', 'акци', 
            'buy', 'order', 'hurry', 'discount'
        ])
        
        # Загрузка эмодзи из Unicode категорий
        self._load_emoji_ranges()
        
    def _load_emoji_ranges(self):
        """Загрузка диапазонов эмодзи из Unicode стандарта"""
        # Используем стандартные Unicode категории для эмодзи
        # Это более надежно, чем хардкодить диапазоны
        self.emoji_categories = {
            'Emoticons': (0x1F600, 0x1F64F),
            'Miscellaneous Symbols': (0x1F300, 0x1F5FF),
            'Transport and Map': (0x1F680, 0x1F6FF),
            'Supplemental Symbols': (0x1F900, 0x1F9FF),
        }
        
    @lru_cache(maxsize=1024)
    def _has_emoji(self, text: str) -> bool:
        """Проверка наличия эмодзи (с кэшированием)"""
        for char in text:
            # Проверяем Unicode категорию символа
            if unicodedata.category(char) in ('So', 'Sk'):  # Symbol, Other / Symbol, Modifier
                code = ord(char)
                for start, end in self.emoji_categories.values():
                    if start <= code <= end:
                        return True
        return False
    
    @lru_cache(maxsize=512)
    def _clean_html_cached(self, text: str) -> str:
        """Кэшированная очистка HTML"""
        return self.patterns['clean_html'].sub(' ', text)
    
    def extract_features(self, text: str, brand_name: Optional[str] = None,
                         category: Optional[str] = None) -> Dict[str, float]:
        """
        Оптимизированное извлечение признаков
        """
        # Конвертируем в строку один раз
        text_str = str(text) if pd.notna(text) else ''
        
        if not text_str:
            return self._get_empty_features()
        
        # Базовая обработка (кэшируем)
        text_clean = self._clean_html_cached(text_str)
        text_lower = text_clean.lower()
        
        # Параллельное извлечение признаков
        features = {}
        
        # Используем numpy для векторизованных операций где возможно
        features.update(self._extract_html_features_optimized(text_str))
        features.update(self._extract_keyword_features_optimized(text_lower))
        features.update(self._extract_style_features_optimized(text_clean))
        features.update(self._extract_repetition_features_optimized(text_clean))
        
        return features
    
    def _get_empty_features(self) -> Dict[str, float]:
        """Возвращает словарь с нулевыми значениями для пустого текста"""
        return {
            # HTML features
            'has_html_tags': 0, 'html_total_tags': 0, 'html_br_count': 0,
            'html_formatting_tags': 0, 'excessive_formatting': 0,
            'html_suspicious_pattern': 0, 'poor_html_quality': 0,
            
            # Keyword features
            'copy_words_count': 0, 'has_copy_words': 0,
            'price_mentioned': 0, 'call_to_action_count': 0,
            
            # Style features
            'uppercase_ratio': 0, 'punctuation_density': 0,
            'exclamation_marks': 0, 'has_emoji': 0,
            
            # Repetition features
            'word_uniqueness': 0, 'repeated_words_count': 0,
            'text_entropy': 0, 'repeated_sentences': 0
        }
    
    def _extract_html_features_optimized(self, text: str) -> Dict[str, float]:
        """Оптимизированное извлечение HTML признаков"""
        features = {}
        
        # Используем предкомпилированные паттерны
        # Считаем все за один проход
        tag_counts = {}
        for name, pattern in self.patterns.items():
            if name.endswith('_tags') or name.endswith('_attrs'):
                tag_counts[name] = len(pattern.findall(text))
        
        # Базовые признаки
        features['has_html_tags'] = 1 if '<' in text else 0
        features['html_br_count'] = tag_counts.get('br_tags', 0)
        features['html_p_count'] = tag_counts.get('p_tags', 0)
        features['html_div_count'] = tag_counts.get('div_tags', 0)
        features['html_span_count'] = tag_counts.get('span_tags', 0)
        
        # Форматирование
        formatting = (tag_counts.get('strong_tags', 0) + 
                     tag_counts.get('italic_tags', 0) + 
                     tag_counts.get('underline_tags', 0))
        features['html_formatting_tags'] = formatting
        features['excessive_formatting'] = 1 if formatting > 5 else 0
        
        # Стили
        features['html_font_tags'] = tag_counts.get('font_tags', 0)
        features['html_color_usage'] = tag_counts.get('color_attrs', 0)
        features['has_colored_text'] = 1 if tag_counts.get('color_attrs', 0) > 0 else 0
        
        # Заголовки
        features['html_header_tags'] = tag_counts.get('h_tags', 0)
        features['has_h1_tag'] = 1 if tag_counts.get('h1_tags', 0) > 0 else 0
        
        # Entities
        features['html_entities_count'] = tag_counts.get('html_entities', 0)
        features['html_nbsp_count'] = text.count('&nbsp;')
        features['excessive_nbsp'] = 1 if features['html_nbsp_count'] > 10 else 0
        
        # Качество
        features['html_broken_tags'] = tag_counts.get('broken_tags', 0)
        features['poor_html_quality'] = 1 if features['html_broken_tags'] > 0 else 0
        
        # Подозрительные паттерны (вычисляем один раз)
        suspicious = 0
        suspicious += 1 if self.patterns['multiple_br'].search(text) else 0
        suspicious += 1 if self.patterns['multiple_bold'].search(text) else 0
        suspicious += 1 if self.patterns['bright_colors'].search(text) else 0
        suspicious += 2 if ('mso-' in text or 'MsoNormal' in text) else 0
        
        # Проверка устаревших тегов
        for tag in self.deprecated_tags:
            if f'<{tag}' in text.lower():
                suspicious += 1
        
        features['html_suspicious_pattern'] = suspicious
        
        # Общее количество тегов
        features['html_total_tags'] = sum(v for k, v in tag_counts.items() 
                                         if k.endswith('_tags'))
        
        # Соотношения
        text_clean = self._clean_html_cached(text)
        features['html_to_text_ratio'] = 1 - (len(text_clean) / max(len(text), 1))
        features['formatting_density'] = formatting / max(len(text.split()), 1)
        
        return features
    
   
    
    def _extract_keyword_features_optimized(self, text_lower: str) -> Dict[str, float]:
        """Оптимизированное извлечение ключевых слов"""
        features = {}
        
        # Используем frozenset для быстрой проверки вхождения
        copy_score = sum(1 for keyword in self.copy_keywords if keyword in text_lower)
        features['copy_words_count'] = copy_score
        features['has_copy_words'] = 1 if copy_score > 0 else 0
        
        # Цены
        prices = self.patterns['price'].findall(text_lower)
        features['price_mentioned'] = 1 if prices else 0
        features['price_mentions_count'] = len(prices)
        
        # CTA фразы
        features['call_to_action_count'] = sum(1 for phrase in self.cta_phrases 
                                              if phrase in text_lower)
        
        return features
    
    def _extract_style_features_optimized(self, text: str) -> Dict[str, float]:
        """Оптимизированное извлечение стилистических признаков"""
        features = {}
        
        if not text:
            return features
        
        # Векторизованный подсчет символов
        text_array = np.array(list(text))
        
        # Используем numpy для быстрого подсчета
        upper_mask = np.array([c.isupper() for c in text])
        lower_mask = np.array([c.islower() for c in text])
        
        upper_chars = np.sum(upper_mask)
        lower_chars = np.sum(lower_mask)
        total_alpha = upper_chars + lower_chars
        
        features['uppercase_ratio'] = upper_chars / max(total_alpha, 1)
        features['all_uppercase'] = 1 if upper_chars > 0 and lower_chars == 0 else 0
        features['all_lowercase'] = 1 if lower_chars > 0 and upper_chars == 0 else 0
        
        # Капитализация слов
        words = text.split()
        if words:
            capitalized = sum(1 for w in words if w and w[0].isupper())
            features['capitalized_words_ratio'] = capitalized / len(words)
        
        # Пунктуация (используем set для быстрой проверки)
        punct_set = set('!?.,:;-—')
        punct_count = sum(1 for c in text if c in punct_set)
        features['punctuation_density'] = punct_count / max(len(text), 1)
        features['exclamation_marks'] = text.count('!')
        features['question_marks'] = text.count('?')
        features['ellipsis_count'] = text.count('...')
        
        # Эмодзи (используем оптимизированную функцию)
        features['has_emoji'] = 1 if self._has_emoji(text[:100]) else 0  # Проверяем только начало
        
        # Абзацы
        paragraphs = text.split('\n\n')
        features['paragraph_count'] = len(paragraphs)
        if paragraphs:
            lengths = [len(p) for p in paragraphs]
            features['avg_paragraph_length'] = np.mean(lengths)
        else:
            features['avg_paragraph_length'] = 0
        
        return features
    
    def _extract_repetition_features_optimized(self, text: str) -> Dict[str, float]:
        """Оптимизированное извлечение признаков повторений"""
        features = {}
        
        words = text.lower().split()
        
        if not words:
            return {
                'word_uniqueness': 0,
                'repeated_words_count': 0,
                'repeated_bigrams_count': 0,
                'max_word_repetition': 0,
                'text_entropy': 0,
                'repeated_sentences': 0
            }
        
        # Используем Counter для эффективного подсчета
        word_counts = Counter(words)
        
        # Уникальность
        unique_words = len(word_counts)
        total_words = len(words)
        features['word_uniqueness'] = unique_words / total_words
        
        # Повторения
        repeated = [w for w, c in word_counts.items() if c > 2]
        features['repeated_words_count'] = len(repeated)
        
        # Максимальное повторение
        features['max_word_repetition'] = max(word_counts.values())
        
        # Биграммы (оптимизированно)
        if len(words) > 1:
            bigrams = [f"{words[i]} {words[i+1]}" for i in range(len(words)-1)]
            bigram_counts = Counter(bigrams)
            repeated_bigrams = sum(1 for c in bigram_counts.values() if c > 1)
            features['repeated_bigrams_count'] = repeated_bigrams
        else:
            features['repeated_bigrams_count'] = 0
        
        # Энтропия (векторизованный расчет)
        probs = np.array(list(word_counts.values())) / total_words
        features['text_entropy'] = -np.sum(probs * np.log2(probs + 1e-10))
        
        # Повторяющиеся предложения
        sentences = self.patterns['sentences'].split(text)
        sentence_counts = Counter(s.strip() for s in sentences if s.strip())
        features['repeated_sentences'] = sum(1 for c in sentence_counts.values() if c > 1)
        
        return features
    
    def extract_features_batch(self, texts: pd.Series, 
                              brand_names: Optional[pd.Series] = None,
                              categories: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Батчевое извлечение признаков для ускорения
        """
        features_list = []
        
        for i, text in enumerate(texts):
            brand = brand_names.iloc[i] if brand_names is not None else None
            category = categories.iloc[i] if categories is not None else None
            
            features = self.extract_features(text, brand, category)
            features_list.append(features)
        
        return pd.DataFrame(features_list)

In [10]:
data_val['description'] = data_val['description'].fillna('')
data_val['brand_name'] = data_val['brand_name'].fillna('Unknown')

data_raw['description'] = data_raw['description'].fillna('')
data_raw['brand_name'] = data_raw['brand_name'].fillna('Unknown')

/var/folders/x1/d4wrwp8x1cdgx44hq2rdb2xr0000gn/T/ipykernel_9117/3606322630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val['description'] = data_val['description'].fillna('')
/var/folders/x1/d4wrwp8x1cdgx44hq2rdb2xr0000gn/T/ipykernel_9117/3606322630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val['brand_name'] = data_val['brand_name'].fillna('Unknown')
/var/folders/x1/d4wrwp8x1cdgx44hq2rdb2xr0000gn/T/ipykernel_9117/3606322630.py:4: SettingWithCopyWarning: 
A value is trying to be s

In [170]:
data_val

,id,description,brand_name,CommercialTypeName4
0,17384,Приветствую! Мы собираем классные системные бл...,Unknown,Настольный компьютер
1,260316,<li>Тип товара Новый</li><li><b>Тип</b> Картри...,Canon,"Картридж, чернила, тонер"
2,10610,Электронное перо Touch S-Pen для Samsung Galax...,Devia,Стилус
3,205236,Мультиварка GALAXY GL2643 имеет все необходимы...,GALAXY LINE,"Мультиварка, скороварка"
4,308655,Чайники с ретро-термометром позволяет нагреть ...,MAUNFELD,Чайник электрический
...,...,...,...,...
22755,423002,Уникальный силиконовый 3D чехол для Этот гадже...,Unknown,Чехол для портативной акустики
22756,316386,,GARLYN,Пылесос вертикальный
22757,245410,Dell Precision T3420 — Компактная рабочая стан...,Dell,Настольный компьютер
22758,440579,Кабель оптический для телевизора Samsung Q900R...,Unknown,Запчасти для телевизора


In [11]:
extractor = OptimizedEcommerceExtractor()

In [12]:

features_df = extractor.extract_features_batch(
    data_raw['description'],
            
            
)
    
data_raw_with_features = pd.concat([data_raw, features_df.add_prefix('desc_')], axis=1)

In [13]:
features_df = extractor.extract_features_batch(
    data_val['description'],
            
            
)
    
data_val_with_features = pd.concat([data_val, features_df.add_prefix('desc_')], axis=1)

In [215]:
data_raw_with_features

,id,description,brand_name,CommercialTypeName4,resolution,desc_has_html_tags,desc_html_br_count,desc_html_p_count,desc_html_div_count,desc_html_span_count,...,desc_ellipsis_count,desc_has_emoji,desc_paragraph_count,desc_avg_paragraph_length,desc_word_uniqueness,desc_repeated_words_count,desc_max_word_repetition,desc_repeated_bigrams_count,desc_text_entropy,desc_repeated_sentences
0,159385,"Мешки пылесборники для пылесоса PHILIPS, 10 шт...",ACTRUM,Пылесборник,0,0,0,0.0,0.0,0.0,...,0.0,0,1.0,886.0,0.650000,1,34.0,0.0,4.874119,0
1,288616,Защитная силиконовая крышка обьектива GoPro He...,Red Line,Крышка для объектива,0,1,2,0.0,0.0,0.0,...,0.0,0,1.0,388.0,0.867925,1,3.0,0.0,5.449526,0
2,108090,Плоский медиатор из кости толщиной 0.6 мм<br/>...,Talwar Brothers,Аксессуар для музыкального инструмента,0,1,1,0.0,0.0,0.0,...,0.0,0,1.0,218.0,0.757576,1,3.0,4.0,4.536670,0
3,415607,"Игра Sonic Frontiers для PlayStation 5, русски...",Unknown,Видеоигра,0,0,0,0.0,0.0,0.0,...,0.0,0,5.0,111.8,0.913043,1,3.0,0.0,5.923671,0
4,332391,Disney Classic Games: Aladdin and The Lion Kin...,Unknown,Видеоигра,0,0,0,0.0,0.0,0.0,...,0.0,0,1.0,540.0,0.871429,1,3.0,1.0,5.861356,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,159865,Парогенератор Tefal Express Vision SV8152E0 об...,Tefal,Паровая станция,0,0,0,0.0,0.0,0.0,...,0.0,0,1.0,582.0,0.880952,1,3.0,0.0,6.145235,0
197194,299636,Выпрямитель для волос Dyson Airstrait<br/><br/...,Dyson,Выпрямитель для волос Dyson,0,1,10,0.0,0.0,0.0,...,0.0,0,1.0,279.0,0.911765,0,2.0,0.0,4.910992,0
197195,24145,Настольная компактная посудомоечная машина Wei...,Weissgauff,Посудомоечная машина компактная,0,1,2,0.0,0.0,0.0,...,0.0,0,1.0,546.0,0.882353,2,4.0,2.0,5.793345,0
197196,51939,Беспроводные наушники Redmi Buds 4 Lite Молоде...,Xiaomi,Наушники TWS,1,1,6,0.0,0.0,0.0,...,0.0,0,1.0,584.0,0.866667,1,3.0,3.0,5.952087,0


In [79]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer


/Users/vladislavdemin/PycharmProjects/scientificProject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [139]:
data_val_with_features.to_csv('/Users/vladislavdemin/Desktop/ml_hack/data_val_with_features.csv')

In [14]:
from src import text_vectoryzer as tm
import importlib
importlib.reload(tm)

/Users/vladislavdemin/PycharmProjects/scientificProject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'src.tetx_model2' from '/Users/vladislavdemin/PycharmProjects/BB-E-CUP/src/tetx_model2.py'>

In [15]:
vectorizer = tm.CPUOptimizedVectorizer(strategy='fast')

Initialized TF-IDF + SVD (fast mode)


In [143]:
vectorizer.process_large_file(
    file_path='/Users/vladislavdemin/Desktop/ml_hack/data_val_with_features.csv',
    text_column='description',
    chunksize=50000,
    save_dir='/Users/vladislavdemin/Desktop/ml_hack/'
)

Processing file: /Users/vladislavdemin/Desktop/ml_hack/data_val_with_features.csv
File format: .csv
Save format: csv


Processing chunks: 0it [00:00, ?it/s]
Encoding: 100%|██████████| 228/228 [00:04<00:00, 55.93it/s]
Processing chunks: 1it [00:05,  5.95s/it]


Saved chunk 0 to /Users/vladislavdemin/Desktop/ml_hack/embeddings_chunk_0000.csv
Processed 1 chunks. Saved to /Users/vladislavdemin/Desktop/ml_hack/
Merging 1 chunks...


Reading chunks: 100%|██████████| 1/1 [00:00<00:00,  5.01it/s]


Concatenating chunks...
Final embeddings saved to /Users/vladislavdemin/Desktop/ml_hack/embeddings_final.csv
Shape: (22760, 101)
Cleaning up chunk files...


In [122]:
sample_text = data_raw_with_features['description'].iloc[6]
print(f"Original text: {sample_text[:200]}...")

cleaned = vectorizer.clean_text(sample_text)
print(f"Cleaned text: {cleaned[:200]}...")

Original text: Нож саблевидный для мясорубки Philips HR2724, HR2725, HR2726/90, HR2727, HR2728, HR2729, HR7752, HR7754, HR7755, HR7768<br/><br/>Этот нож - точная копия профессиональных ножей используемых в промышлен...
Cleaned text: Нож саблевидный для мясорубки Philips HR2724, HR2725, HR2726/90, HR2727, HR2728, HR2729, HR7752, HR7754, HR7755, HR7768 Этот нож - точная копия профессиональных ножей используемых в промышленных мясор...


In [123]:
embedding = vectorizer.encode(cleaned, show_progress=False)
print(f"Embedding shape: {embedding.shape}")

Embedding shape: (1, 384)


In [16]:
chunk_0000 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/chunk_train/embeddings_chunk_0000.csv')

In [17]:
chunk_0001 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/chunk_train/embeddings_chunk_0001.csv')

In [18]:
chunk_0002 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/chunk_train/embeddings_chunk_0002.csv')

In [19]:
chunk_0003 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/chunk_train/embeddings_chunk_0003.csv')

In [20]:
all_chuncks = pd.concat([chunk_0000, chunk_0001, chunk_0002, chunk_0003], axis=0)

In [21]:
chunk_test = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/embeddings_final.csv')

In [22]:
all_chuncks = all_chuncks.reset_index(drop=True)

In [23]:
data_raw_with_features_and_embed = pd.concat([data_raw_with_features, all_chuncks], axis=1)

In [24]:
data_raw_with_features_and_embed = data_raw_with_features_and_embed.drop('description', axis=1)

In [25]:
data_raw_with_features_and_embed = data_raw_with_features_and_embed.drop('id', axis=1)

In [26]:
chunk_test = chunk_test.reset_index(drop=True)

In [27]:
data_val_with_features_and_embed = pd.concat([data_val_with_features, chunk_test], axis=1)

In [29]:
data_val_with_features_and_embed = data_val_with_features_and_embed.drop('description', axis=1)

KeyError: "['description'] not found in axis"

In [30]:
data_val_with_features_and_embed = data_val_with_features_and_embed.drop('id', axis=1)

In [29]:
cols_with_unknown = data_raw_with_features_and_embed.columns[data_raw_with_features_and_embed.astype(str).apply(lambda col: col.str.contains("Unknow", na=False)).any()]

In [152]:
cols_with_unknown_val = data_val_with_features_and_embed.columns[data_val_with_features_and_embed.astype(str).apply(lambda col: col.str.contains("Unknow", na=False)).any()]

In [32]:
data_raw_with_features_and_embed = data_raw_with_features_and_embed.replace('Unknow', np.nan)
data_raw_with_features_and_embed = data_raw_with_features_and_embed.replace('Unknown', np.nan)

In [34]:
data_raw_with_features_and_embed['brand_name'] = data_raw_with_features_and_embed['brand_name'].fillna('Unknown')

In [31]:
brand_counts = data_raw_with_features_and_embed["brand_name"].value_counts()

# Все бренды, которые встречаются меньше 10 раз → 'Unknown'
data_raw_with_features_and_embed["brand_name"] = data_raw_with_features_and_embed["brand_name"].where(data_raw_with_features_and_embed["brand_name"].map(brand_counts) >= 15, "Unknown")

CommercialTypeName4_counts = data_raw_with_features_and_embed["CommercialTypeName4"].value_counts()

# Все бренды, которые встречаются меньше 10 раз → 'Unknown'
data_raw_with_features_and_embed["CommercialTypeName4"] = data_raw_with_features_and_embed["CommercialTypeName4"].where(data_raw_with_features_and_embed["CommercialTypeName4"].map(CommercialTypeName4_counts) >= 15, "Unknown")




brand_counts_val = data_val_with_features_and_embed["brand_name"].value_counts()

# Все бренды, которые встречаются меньше 10 раз → 'Unknown'
data_val_with_features_and_embed["brand_name"] = data_val_with_features_and_embed["brand_name"].where(data_val_with_features_and_embed["brand_name"].map(brand_counts) >= 15, "Unknown")

CommercialTypeName4_counts_val = data_val_with_features_and_embed["CommercialTypeName4"].value_counts()

# Все бренды, которые встречаются меньше 10 раз → 'Unknown'
data_val_with_features_and_embed["CommercialTypeName4"] = data_val_with_features_and_embed["CommercialTypeName4"].where(data_val_with_features_and_embed["CommercialTypeName4"].map(CommercialTypeName4_counts) >= 15, "Unknown")





In [32]:
data_raw_with_features_and_embed['CommercialTypeName4'].value_counts()

CommercialTypeName4
Дисплеи для телефонов                  15002
Аккумулятор для мобильного телефона    11965
Корпуса для телефонов                  11960
Запчасти для телевизора                10192
Картридж, чернила, тонер                8570
                                       ...  
Звуковая карта для ПК                     15
Наклейка для планшета                     15
Кнопка спуска затвора для фото            15
Запчасти для вытяжки                      15
Медиаплеер                                15
Name: count, Length: 377, dtype: int64

In [33]:
all_data = pd.concat([data_raw_with_features_and_embed, data_val_with_features_and_embed], axis=0)

all_dummies = pd.get_dummies(all_data, columns=["brand_name", "CommercialTypeName4"], dummy_na=True, drop_first=False)

data_raw_with_features_and_embed_OHE = all_dummies.iloc[:len(data_raw_with_features)]
data_val_with_features_and_embed_OHE  = all_dummies.iloc[len(data_raw_with_features):]

In [34]:
data_val_with_features_and_embed_OHE = data_val_with_features_and_embed_OHE.drop('resolution', axis=1)

In [238]:
data_val_with_features_and_embed_OHE

,desc_has_html_tags,desc_html_br_count,desc_html_p_count,desc_html_div_count,desc_html_span_count,desc_html_formatting_tags,desc_excessive_formatting,desc_html_font_tags,desc_html_color_usage,desc_has_colored_text,...,"CommercialTypeName4_Штатив, монопод для смартфона",CommercialTypeName4_Щипцы для волос электрические,CommercialTypeName4_Экшен камера,CommercialTypeName4_Электрическая зубная щетка,CommercialTypeName4_Электрогриль,CommercialTypeName4_Электронные компоненты,CommercialTypeName4_Электронные компоненты-запчасти,CommercialTypeName4_Электронный замок,CommercialTypeName4_Электронный модуль,CommercialTypeName4_nan
0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,1,0,0.0,0.0,0.0,17,1,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,1,4,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,1,8,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22755,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
22756,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
22757,1,2,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
22758,1,8,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [35]:
data_raw_full = data_raw_full.drop(['description'], axis=1)
data_raw_full = data_raw_full.fillna(0)

data_val_full = data_val_full.drop(['description'], axis=1)
data_val_full = data_val_full.fillna(0)

In [36]:
data_raw_full = data_raw_full.reset_index(drop=True)
data_val_full = data_val_full.reset_index(drop=True)

In [37]:
data_raw_with_features_and_embed_OHE_with_metadata = pd.concat([data_raw_with_features_and_embed_OHE, data_raw_full], axis=1)
data_val_with_features_and_embed_OHE_with_metadata = pd.concat([data_val_with_features_and_embed_OHE, data_val_full], axis=1)


In [247]:
indexes = [i for i, col in enumerate(data_raw_with_features_and_embed_OHE_with_metadata.columns) if col == "resolution"]
print(indexes)

[0, 1180]


In [293]:
data_raw_with_features_and_embed_OHE_with_metadata1 = data_raw_with_features_and_embed_OHE_with_metadata.iloc[:, 1:]
#не работает

In [294]:
data_raw_with_features_and_embed_OHE_with_metadata1

,desc_html_br_count,desc_html_p_count,desc_html_div_count,desc_html_span_count,desc_html_formatting_tags,desc_excessive_formatting,desc_html_font_tags,desc_html_color_usage,desc_has_colored_text,desc_html_header_tags,...,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID,resolution
0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,50.0,730.171845,896.528847,1043.118191,1.0,1.0,1860.0,78312,1218,0
1,2,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,54.0,993.043882,1137.421611,1188.608000,1.0,1.0,1757.0,141999,1374,0
2,1,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,34.0,800.822138,1174.069505,1224.798286,1.0,1.0,1722.0,53306,1448,0
3,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,6.0,0.000000,913.530121,982.789171,3.0,3.0,1692.0,202599,715,0
4,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,6.0,0.000000,913.542170,982.783783,3.0,3.0,1692.0,163725,715,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,3287.0,1411.575785,1562.870338,1659.098176,1.0,1.0,617.0,78564,1425,0
197194,10,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,14.0,1209.998085,1304.055594,1360.362870,1.0,1.0,789.0,147429,1220,0
197195,2,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,382.0,1339.132808,1477.981760,1582.208540,1.0,1.0,533.0,12216,3188,0
197196,6,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,407.0,1001.608687,1163.593791,1289.893002,1.0,1.0,295.0,25909,4812,1


In [39]:
data_raw_with_features_and_embed_OHE_with_metadata['resolution'] = data_raw_with_features['resolution']

In [40]:
data_raw_with_features_and_embed_OHE_with_metadata = data_raw_with_features_and_embed_OHE_with_metadata.drop(['brand_name', 'name_rus', 'CommercialTypeName4','id'], axis=1)

In [41]:
data_val_with_features_and_embed_OHE_with_metadata = data_val_with_features_and_embed_OHE_with_metadata.drop(['brand_name', 'name_rus', 'CommercialTypeName4','id'], axis=1)

In [222]:
X = data_raw_with_features_and_embed_OHE_with_metadata.drop('resolution', axis=1)
y = data_raw_with_features_and_embed_OHE_with_metadata['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [231]:
from xgboost import XGBClassifier
XGB_model = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=9,
    eval_metric="logloss", use_label_encoder=False,
    class_weight="balanced_subsample" ,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.5)# более мягкий вариант
)
RFC_model = RandomForestClassifier(n_estimators=250, max_depth=10, class_weight="balanced")


In [232]:
XGB_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced_subsample', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None, ...)

In [233]:
y_pred = XGB_model.predict(X_test)

In [234]:
accuracy_RFC_model = accuracy_score(y_test, y_pred),
f1_macro_RFC_model = f1_score(y_test, y_pred, average="macro"),
report_RFC_model = classification_report(y_test, y_pred)

In [235]:
accuracy_RFC_model

(0.9662416690814257,)

In [228]:
print(report_RFC_model)

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     25780
           1       0.76      0.81      0.79      1828

    accuracy                           0.97     27608
   macro avg       0.87      0.90      0.88     27608
weighted avg       0.97      0.97      0.97     27608



In [257]:
list(XGB_model.feature_importances_)

[np.float32(0.00031043548),
 np.float32(0.00041051803),
 np.float32(0.0005333951),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0012220116),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0015618342),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.0014637093),
 np.float32(0.0013342089),
 np.float32(0.0),
 np.float32(0.003862897),
 np.float32(0.0),
 np.float32(0.005828393),
 np.float32(0.0),
 np.float32(0.00043142028),
 np.float32(0.002309777),
 np.float32(0.0),
 np.float32(0.0),
 np.float32(0.007057572),
 np.float32(0.0012743938),
 np.float32(0.006723262),
 np.float32(0.00067784666),
 np.float32(0.0009718832),
 np.float32(0.0),
 np.float32(0.0010987126),
 np.float32(0.0014178564),
 np.float32(0.0008287656),
 np.float32(0.000566448),
 np.float32(0.00028663623),
 np.float32(0.0007183139),
 np.float32(0.0010227545),
 np.float32(0.0),
 np.float32(0.002488

In [258]:
scoring_xgb = pd.DataFrame({
    "feature": list(XGB_model.feature_names_in_),
    'coef':  list(XGB_model.feature_importances_)
})

In [261]:
scoring_xgb.sort_values(by="coef", ascending=False)

,feature,coef
1065,CommercialTypeName4_Планшетный компьютер,0.023933
915,CommercialTypeName4_Дисплеи для телефонов,0.019598
878,CommercialTypeName4_Видеоигра Sony,0.016527
1044,CommercialTypeName4_Настольный компьютер,0.015945
1187,item_time_alive,0.014418
...,...,...
711,brand_name_ZMI,0.000000
712,brand_name_ZOMEi,0.000000
713,brand_name_ZOOM,0.000000
714,brand_name_ZTE,0.000000


In [236]:
val_prediction = XGB_model.predict(data_val_with_features_and_embed_OHE_with_metadata)

In [237]:
val_prediction.sum()

np.int64(804)

In [193]:
submission = pd.DataFrame({
    'id': data_val_full['id'],
    'prediction': val_prediction
    
})

In [194]:
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)

In [195]:
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')

In [196]:
dddd.sum()

id            5060458544
prediction           825
dtype: int64

In [164]:
gs = data1[['CommercialTypeName4', 'brand_name']].fillna('Unknown')

In [165]:
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(gs)

OneHotEncoder(handle_unknown='ignore')

In [167]:
encoder.fit_transform(gs)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 394396 stored elements and shape (197198, 4701)>
  Coords	Values
  (0, 449)	1.0
  (0, 707)	1.0
  (1, 306)	1.0
  (1, 3249)	1.0
  (2, 42)	1.0
  (2, 3729)	1.0
  (3, 115)	1.0
  (3, 3901)	1.0
  (4, 115)	1.0
  (4, 3901)	1.0
  (5, 115)	1.0
  (5, 3901)	1.0
  (6, 201)	1.0
  (6, 3901)	1.0
  (7, 201)	1.0
  (7, 3901)	1.0
  (8, 201)	1.0
  (8, 3901)	1.0
  (9, 19)	1.0
  (9, 4623)	1.0
  (10, 546)	1.0
  (10, 3901)	1.0
  (11, 561)	1.0
  (11, 3901)	1.0
  (12, 68)	1.0
  :	:
  (197185, 2186)	1.0
  (197186, 410)	1.0
  (197186, 2002)	1.0
  (197187, 256)	1.0
  (197187, 3431)	1.0
  (197188, 333)	1.0
  (197188, 4080)	1.0
  (197189, 368)	1.0
  (197189, 2186)	1.0
  (197190, 156)	1.0
  (197190, 3565)	1.0
  (197191, 575)	1.0
  (197191, 4080)	1.0
  (197192, 314)	1.0
  (197192, 4080)	1.0
  (197193, 404)	1.0
  (197193, 3752)	1.0
  (197194, 152)	1.0
  (197194, 1468)	1.0
  (197195, 432)	1.0
  (197195, 4023)	1.0
  (197196, 370)	1.0
  (197196, 4080)	1.0
  (1971

In [295]:
dffff = pd.DataFrame({
    'A' : [1, 2 , 3],
    'B': [4, 5, 6],
    'A' : [7, 8, 9]
})

In [296]:
dffff.drop('A', axis=1)

,B
0,4
1,5
2,6


In [339]:
from src import preproces_text as pt
from src import future_extract as fe 
import importlib
importlib.reload(fe)
from src import text_model as tm

In [305]:
data_name_rus_raw = data_raw_full['name_rus']
data_name_rus_val = data_val_full['name_rus']

In [332]:
extractor = fe.OptimizedEcommerceExtractor()

columns_for_del = ['has_html_tags', 'html_br_count', 'html_p_count', 'html_div_count', 'html_span_count', 'html_formatting_tags', 'excessive_formatting', 'html_font_tags', 'html_color_usage', 'has_colored_text','html_header_tags','has_h1_tag', 'html_entities_count', 'html_nbsp_count', 'excessive_nbsp', 'html_broken_tags', 'poor_html_quality', 'html_suspicious_pattern', 'html_total_tags', 'html_to_text_ratio', 'formatting_density', 'question_marks', 'has_emoji', 'paragraph_count']


features_df_name_rus_raw = extractor.extract_features_batch(data_name_rus_raw)
features_df_name_rus_raw = features_df_name_rus_raw.drop(columns_for_del, axis=1)
name_rus_raw_with_features = pd.concat([data_name_rus_raw, features_df_name_rus_raw.add_prefix('desc_')], axis=1)


features_df_name_rus_val = extractor.extract_features_batch(data_name_rus_val)
features_df_name_rus_val = features_df_name_rus_val.drop(columns_for_del, axis=1)
name_rus_val_with_features = pd.concat([data_name_rus_val, features_df_name_rus_val.add_prefix('desc_')], axis=1)


In [345]:
vectorizer = tm.CPUOptimizedVectorizer(strategy='fast')



name_rus_raw_with_features_and_embed = vectorizer.process_dataframe(
             name_rus_raw_with_features,
             text_column='name_rus',
             batch_size=1000,
             add_to_original=True  # Добавить к исходным данным
         )

name_rus_val_with_features_and_embed = vectorizer.process_dataframe(
        name_rus_val_with_features,
        text_column='name_rus',
        batch_size=1000,
        add_to_original=True  # Добавить к исходным данным
    )

name_rus_raw_with_features_and_embed = name_rus_raw_with_features_and_embed.drop(['name_rus'], axis=1)
name_rus_val_with_features_and_embed = name_rus_val_with_features_and_embed.drop(['name_rus'], axis=1)

   

Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 198/198 [00:07<00:00, 27.21it/s]


Created 300 embedding columns
Final DataFrame shape: (197198, 320)
Processing DataFrame with 22760 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 23/23 [00:00<00:00, 31.11it/s]

Created 300 embedding columns
Final DataFrame shape: (22760, 320)


In [347]:
name_rus_val_with_features

,name_rus,desc_copy_words_count,desc_has_copy_words,desc_price_mentioned,desc_price_mentions_count,desc_call_to_action_count,desc_uppercase_ratio,desc_all_uppercase,desc_all_lowercase,desc_capitalized_words_ratio,desc_punctuation_density,desc_exclamation_marks,desc_ellipsis_count,desc_avg_paragraph_length,desc_word_uniqueness,desc_repeated_words_count,desc_max_word_repetition,desc_repeated_bigrams_count,desc_text_entropy,desc_repeated_sentences
0,"LeBefane Системный блок (AMD Ryzen 5 7500F, RA...",0,0,0,0,0,0.474359,0,0,0.583333,0.038760,0,0,129.0,0.958333,0,2,0,4.501629,0
1,Картридж лазерный Canon 071H 5646C002 черный (...,0,0,0,0,0,0.300000,0,0,0.416667,0.020000,0,0,100.0,0.916667,0,2,0,3.418296,0
2,Стилус для Samsung Galaxy Z Fold5 5G/ SM-F946B...,0,0,0,0,0,0.421053,0,0,0.777778,0.050847,0,0,59.0,1.000000,0,1,0,3.169925,0
3,"Мультиварка GALAXY GL2643 ( 900 Вт, 10 програм...",0,0,0,0,0,0.123596,0,0,0.250000,0.039683,0,0,126.0,1.000000,0,1,0,4.321928,0
4,Чайник электрический MAUNFELD MGK-625MINT. Тов...,0,0,0,0,0,0.354167,0,0,0.666667,0.034483,0,0,58.0,1.000000,0,1,0,2.584962,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22755,Защитные наушники Huawei Free Clip,0,0,0,0,0,0.133333,0,0,0.800000,0.000000,0,0,34.0,1.000000,0,1,0,2.321928,0
22756,Пылесос вертикальный беспроводной GARLYN M-750...,0,0,0,0,0,0.214286,0,0,0.666667,0.019231,0,0,52.0,1.000000,0,1,0,2.584962,0
22757,Dell Системный блок производительная рабочая с...,0,0,0,0,0,0.278351,0,0,0.565217,0.051948,0,0,154.0,0.956522,0,2,0,4.436605,0
22758,Кабель оптический для телевизора Samsung Q900R...,0,0,0,0,0,0.170732,0,0,0.571429,0.017241,0,0,58.0,1.000000,0,1,0,2.807355,0


In [343]:
data_raw_ready = pd.concat([data_raw_with_features_and_embed_OHE_with_metadata, name_rus_raw_with_features_and_embed], axis=1)
data_val_ready = pd.concat([data_val_with_features_and_embed_OHE_with_metadata, name_rus_val_with_features_and_embed], axis=1)


In [344]:
data_raw_ready

,desc_has_html_tags,desc_html_br_count,desc_html_p_count,desc_html_div_count,desc_html_span_count,desc_html_formatting_tags,desc_excessive_formatting,desc_html_font_tags,desc_html_color_usage,desc_has_colored_text,...,embed_290,embed_291,embed_292,embed_293,embed_294,embed_295,embed_296,embed_297,embed_298,embed_299
0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,-0.013914,0.000928,0.031865,0.044679,0.009562,0.035227,0.009949,-0.011175,-0.028408,-0.004356
1,1,2,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,-0.024317,-0.085667,0.004619,-0.032996,0.040596,-0.016709,0.051361,-0.005043,-0.043541,-0.016947
2,1,1,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.013109,0.005287,-0.001031,-0.008884,0.018046,-0.004072,0.010311,0.005618,-0.002219,0.002644
3,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.006766,0.021159,-0.003646,-0.019739,-0.025492,-0.017155,-0.015340,0.000513,0.048263,-0.004055
4,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.011933,-0.013774,0.009561,0.035600,0.047425,-0.033439,-0.006617,0.022005,0.021142,0.013282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.034038,0.035568,-0.068424,0.019331,0.034556,0.008834,-0.025235,0.004089,0.015408,-0.020495
197194,1,10,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
197195,1,2,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.061473,0.010142,0.002087,-0.001306,-0.003187,0.009710,-0.033004,0.021497,-0.011469,0.022383
197196,1,6,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,-0.042474,0.026060,0.049329,-0.008034,0.018174,-0.019133,0.004038,-0.052416,-0.021058,0.015040
